## Large scale NAIP mosaic

This notebook aims to show how we can use dynamoDB backends to store large scale mosaic (US wide) for large scale dataset as NAIP.

Checkout previous work by Kyle Barron: https://github.com/kylebarron/naip-cogeo-mosaic

In [ ]:
import requests
from ipyleaflet import Map, basemaps, TileLayer, basemap_to_tiles, GeoJSON

from cogeo_mosaic.backends import MosaicBackend

In [ ]:
# First we need to create a Mosaic
# Thanks to Kyle Barron, most of the job is already done. We just need to download one of his mosaic and upload it to our DynamoDB Table
url = "https://raw.githubusercontent.com/kylebarron/naip-cogeo-mosaic/master/data/naip_2016_2018_mosaic.json.gz"

# Important, we are using a new version of cogeo-mosaic to store and read mosaicJSON documents (>=3.0.0a15)
# See: https://github.com/developmentseed/cogeo-mosaic/pull/127
#
# Because the Naip mosaic is > 60Mb, it's way more efficient to use dynamoDB backend.
# Make sure your stack is configure with the right info
# e.g
# $ cat stack/.env | grep "STACK_MOSAIC"
# STACK_MOSAIC_BACKEND=dynamodb://
# STACK_MOSAIC_HOST=us-west-2/mymosaictable

# Name of the mosaic (== layer in titiler)
mosaic_name = "naip.2016_2018"

# EDIT THIS. by default cogeo-mosaic will create the DynamoDB table (billing mode set to PAY_PER_REQUEST).
# You can also use a pre-created table of yours as long that it follows the schema:
# https://github.com/developmentseed/cogeo-mosaic/blob/newDynamoDBschema/cogeo_mosaic/backends/dynamodb.py#L182-L191
table_name = "mymosaictable"

# NAIP, Sentinel-2 and Landsat is stored in us-west-2, so it makes sense to deploy titiler in us-west-2 and also to store the dynamoDB table there
region = "us-west-2"

# 1. Read mosaic json from Kyle
with MosaicBackend(url) as src_mosaic:
    
    # 2. use MosaicBackend to write the mosaic to dynamoDB
    with MosaicBackend(
        f"dynamodb://{region}{table_name}:{mosaic_name}",
        mosaic_def=src_mosaic.mosaic_def
    ) as dst_mosaic:
        dst_mosaic.write()

In [ ]:
# Endpoint variables
titiler_endpoint = "{ENDPOINT}"

# Make sure both are up
assert requests.get(f"{titiler_endpoint}/docs").status_code == 200

In [ ]:
r = requests.get(f"{titiler_endpoint}/mosaicjson/naip/info?layer={mosaic_name}").json()
print(r)

In [ ]:
r = requests.get(f"{titiler_endpoint}/mosaicjson/naip/tilejson.json?layer={mosaic_name}&bidx=1,2,3").json()
print(r)

In [ ]:
bounds = r["bounds"]
m = Map(
    center=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom=r["minzoom"]
)

tiles = TileLayer(
    url=r["tiles"][0],
    min_zoom=r["minzoom"],
    max_zoom=r["maxzoom"],
    opacity=1
)
m.add_layer(tiles)
m